파이썬 머신러닝 완벽 가이드 ch9.6-8 591p~ 624p

---

## 6. 아이템 기반 최근접 이웃 협업 필터링 실습

사용자-영화 평점 행렬 데이터 셋 이용

- 데이터 가공 및 변환

In [1]:
import pandas as pd
import numpy as np

movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
print(movies.shape)
print(ratings.shape)

(9742, 3)
(85271, 4)


In [2]:
ratings = ratings[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')
ratings_matrix.head(3)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# title 칼럼을 얻기 위해 movies와 조인
rating_movies = pd.merge(ratings, movies, on='movieId')

# columns='title'로 title 칼럼으로 피벗 수행
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')

# NaN 값을 모두 0으로 변환
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix.head(3)

title,'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...All the Marbles (1981),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
ratings_matrix_T = ratings_matrix.transpose()
ratings_matrix_T.head(3)

userId,1,2,3,4,5,6,7,8,9,10,...,545,546,547,548,549,550,551,552,553,554
title,,,,,,,,,,,,,,,,,,,,,
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
#코사인 유사도
from sklearn.metrics.pairwise import cosine_similarity
item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

# cosine_similarity()로 반환된 넘파이 행렬을 영화명을 매핑해 DataFrame으로 변환
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns, columns=ratings_matrix.columns)
print(item_sim_df.shape)
item_sim_df.head(3)

(8753, 8753)


title,'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...All the Marbles (1981),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'Hellboy': The Seeds of Creation (2004),1.000000,0.707107,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.707107,1.000000,0.0,0.000000,0.0,0.204298,0.0,0.0,0.0,0.707107,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.000000,0.000000,1.0,0.857493,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# 대부와 유사도 높은 상위 6개 영화 추출
item_sim_df["Godfather, The (1972)"].sort_values(ascending=False)[:6]

title
Godfather, The (1972)                     1.000000
Godfather: Part II, The (1974)            0.813752
Goodfellas (1990)                         0.661416
One Flew Over the Cuckoo's Nest (1975)    0.616324
Pulp Fiction (1994)                       0.588956
Fargo (1996)                              0.585011
Name: Godfather, The (1972), dtype: float64

In [7]:
# 인셉션과 유사도 높은 영화 (자신 제외)
item_sim_df["Inception (2010)"].sort_values(ascending=False)[1:6]

title
Dark Knight, The (2008)          0.727626
Inglourious Basterds (2009)      0.631824
Fight Club (1999)                0.608420
Dark Knight Rises, The (2012)    0.603498
Shutter Island (2010)            0.598649
Name: Inception (2010), dtype: float64

In [8]:
def predict_rating(ratings_arr, item_sim_arr):
    ratings_pred = ratings_arr.dot(item_sim_arr) / np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

In [9]:
# 개인화된 예측 평점 구하기
ratings_pred=predict_rating(ratings_matrix.values, item_sim_df.values)
ratings_pred_matrix=pd.DataFrame(data=ratings_pred, index=ratings_matrix.index, columns=ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...All the Marbles (1981),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.580812,0.328981,0.223842,0.208265,0.196484,0.249569,0.103216,0.172198,0.191951,0.123860,...,0.114874,0.223511,0.221029,0.206858,0.006373,0.218616,0.194333,0.166381,0.305049,0.710160
2,0.042309,0.018992,0.000000,0.000000,0.035810,0.010957,0.002299,0.035239,0.019067,0.000000,...,0.015562,0.020688,0.022406,0.011995,0.050745,0.014633,0.021869,0.027229,0.017280,0.000000
3,0.031262,0.064352,0.003578,0.003638,0.002808,0.012122,0.002148,0.005242,0.006945,0.091304,...,0.006996,0.010214,0.011800,0.012783,0.000000,0.008377,0.006526,0.008288,0.008900,0.083829


In [10]:
from sklearn.metrics import mean_squared_error

# 사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 mse를 구함
def get_mse(pred, actual):
    # 평점이 있는 실제 영화만 추출
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 최근접 이웃 mse : ', get_mse(ratings_pred, ratings_matrix.values))

아이템 기반 모든 최근접 이웃 mse :  9.939126477508403


In [11]:
#mse 감소시키는 방향으로 개선 -> 많은 영화 유사도 벡터 이용해 평점 예측 떨어짐
#-> 특정 영화와 가장 비슷한 유사도 가진 영화 가지는 영화에 대해서만 적용

def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape)

    # 사용자-아이템 평점 행렬의 열 크기만큼 루프 수행.
    for col in range(ratings_arr.shape[1]):
        # 유사도 행렬에서 유사도가 큰 순으로 n개 데이터 행렬의 index 반환
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n - 1:-1]]
        # 개인화된 예측 평점을 계산
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T)
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))
    return pred


In [12]:
#예측 평점 계산, 실제 평점과의 mse 구하기
ratings_pred = predict_rating_topsim(ratings_matrix.values, item_sim_df.values, n=20)
print('아이템 기반 최근접 top20 이웃 mse : ', get_mse(ratings_pred, ratings_matrix.values))

# 계산된 예측 평점 데이터는 dataframe으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index, columns=ratings_matrix.columns)

<ipython-input-11-c486719ad3cc>:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T)


아이템 기반 최근접 top20 이웃 mse :  3.8214423867995824


In [13]:
#mse 작아짐 -> 성능 향상, 특정 사용자에 대한 영화 추천해보기
user_rating_id = ratings_matrix.loc[9, :]
user_rating_id[user_rating_id > 0].sort_values(ascending=False)[:10]

title
Adaptation (2002)                                                                 5.0
Citizen Kane (1941)                                                               5.0
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)    5.0
Producers, The (1968)                                                             5.0
Lord of the Rings: The Two Towers, The (2002)                                     5.0
Lord of the Rings: The Fellowship of the Ring, The (2001)                         5.0
Back to the Future (1985)                                                         5.0
Austin Powers in Goldmember (2002)                                                5.0
Minority Report (2002)                                                            4.0
Witness (1985)                                                                    4.0
Name: 9, dtype: float64

In [14]:
# 사용자가 이미 평점 준 영화 제외 추천할 수 있도록 평점 주지 않은 영화 리스트 객체로반환하는 함수
def get_unseen_movie(ratings_matrix, userId):
  # userid로 입력받은 사용자의 모든 영화 정보 추출해 series로 반환함
  # 반환된 user_rating 은 영화명(title)을 index로 가지는 Series 객체임
  user_rating = ratings_matrix.loc[userId, :]

  # user_rating이 0보다 크면 기존에 관람한 영화임. 대상 index를 추출해 list 객체로 만듦
  already_seen = user_rating[user_rating > 0].index.tolist()

  # 모든 영화명을 list 객체로 만듦
  movies_list = ratings_matrix.columns.tolist()

  # list comprehension으로 already_seen에 해당하는 영화는 movies_list에서 제외함
  unseen_list = [movie for movie in movies_list if movie not in already_seen]

  return unseen_list


In [15]:
# 예측 평점 df와 추천하려는 사용자 id, 추천후보 영화 리스트, 추천 상위 영화 개수 인자로 받아서 사용자가 가장 좋아할만한 높은 예측 평점 가진 영화 추천

def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
  # 예측 평점 DataFrame에서 사용자 id 인덱스와 unseen_list로 들어온 영화명 칼럼을 추출해
  # 가장 예측 평점이 높은 순으로 정렬함
  recomm_movie = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
  return recomm_movie

# 사용자가 관람하지 않는 영화명 추출
unseen_list = get_unseen_movie(ratings_matrix, 9)

# 아이템 기반의 인접 이웃 협업 필터링으로 영화 추천
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

# 평점 데이터를 dataframe으로 생성
recomm_movies = pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index, columns=['pred_score'])

recomm_movies

,pred_score
title,
Shrek (2001),0.859526
Spider-Man (2002),0.850298
"Monsters, Inc. (2001)",0.835870
Indiana Jones and the Temple of Doom (1984),0.816420
"Last Samurai, The (2003)",0.810022
"Matrix Reloaded, The (2003)",0.798520
Gladiator (2000),0.746471
"Matrix, The (1999)",0.739986
"Fifth Element, The (1997)",0.700404


## 7. 행렬 분해를 이용한 잠재 요인 협업 필터링 실습


In [16]:
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda = 0.01):
  num_users, num_items = R.shape
  # P와 Q 매트릭스의 크기를 지정하고 정규 분포를 가진 랜덤한 값으로 입력합니다.
  np.random.seed(1)
  P = np.random.normal(scale=1./K, size=(num_users, K))
  Q = np.random.normal(scale=1./K, size=(num_items, K))

  prev_rmse = 10000
  break_count = 0

  # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장
  non_zeros = [(i,j,R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j]>0]

  # SGD기반으로 P와 Q 매트릭스를 계속 업데이트
  for step in range(steps):
    for i, j, r in non_zeros:
      # 실제 값과 예측 값의 차이인 오류값 구함
      eij = r - np.dot(P[i, :], Q[j, :].T)
      # Regularization을 반영한 sgd 업데이트 공식 적용
      P[i, :] += learning_rate * (eij * Q[j, :] - r_lambda * P[i, :])
      Q[j, :] += learning_rate * (eij * P[i, :] - r_lambda * Q[j, :])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 10) == 0 :
      print("### iteration step : ", step, " rmse : ", rmse)

  return  P,Q

In [17]:
def get_rmse(R, P, Q, non_zeros):
  error = 0
  full_pred_matrix = np.dot(P, Q.T)

  x_non_zeros = [non_zero[0] for non_zero in non_zeros]
  y_non_zeros = [non_zero[1] for non_zero in non_zeros]
  R_non_zeros = [non_zero[2] for non_zero in non_zeros]

  full_pred_matrix_non_zeros = full_pred_matrix[x_non_zeros, y_non_zeros]
  R_non_zeros = np.array(R_non_zeros)

  mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
  rmse = np.sqrt(mse)

  return rmse

In [19]:
import pandas as pd
import numpy as np

movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
ratings = ratings[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')
# title 칼럼을 얻기 위해 moview와 조인 수행
rating_movies = pd.merge(ratings, movies, on='movieId')
# columns='title'로 title 칼럼으로 pivot 수행
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')

In [20]:
P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=200, learning_rate=0.01, r_lambda=0.01)
pred_matrix = np.dot(P, Q.T)

### iteration step :  0  rmse :  2.9023619751336867
### iteration step :  10  rmse :  0.7335768591017927
### iteration step :  20  rmse :  0.5115539026853442
### iteration step :  30  rmse :  0.37261628282537446
### iteration step :  40  rmse :  0.2960818299181014
### iteration step :  50  rmse :  0.2520353192341642
### iteration step :  60  rmse :  0.22487503275269854
### iteration step :  70  rmse :  0.2068545530233154
### iteration step :  80  rmse :  0.19413418783028685
### iteration step :  90  rmse :  0.18470082002720406
### iteration step :  100  rmse :  0.17742927527209104
### iteration step :  110  rmse :  0.1716522696470749
### iteration step :  120  rmse :  0.16695181946871726
### iteration step :  130  rmse :  0.16305292191997542
### iteration step :  140  rmse :  0.15976691929679646
### iteration step :  150  rmse :  0.1569598699945732
### iteration step :  160  rmse :  0.15453398186715425
### iteration step :  170  rmse :  0.15241618551077643
### iteration step :  180  rm

In [21]:
ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index=ratings_matrix.index, columns=ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.055084,4.092018,3.564130,4.502167,3.981215,1.271694,3.603274,2.333266,5.091749,3.972454,...,1.402608,4.208382,3.705957,2.720514,2.787331,3.475076,3.253458,2.161087,4.010495,0.859474
2,3.170119,3.657992,3.308707,4.166521,4.311890,1.275469,4.237972,1.900366,3.392859,3.647421,...,0.973811,3.528264,3.361532,2.672535,2.404456,4.232789,2.911602,1.634576,4.135735,0.725684
3,2.307073,1.658853,1.443538,2.208859,2.229486,0.780760,1.997043,0.924908,2.970700,2.551446,...,0.520354,1.709494,2.281596,1.782833,1.635173,1.323276,2.887580,1.042618,2.293890,0.396941


In [22]:
# 사용자가 관람하지 않은 영화명 추출
unseen_list = get_unseen_movie(ratings_matrix, 9)

# 잠재 요인 협업 필터링으로 영화 추천
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

# 평점 데이터를 dataframe으로 생성
recomm_movies = pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index, columns=['pred_score'])
recomm_movies

,pred_score
title,
Rear Window (1954),5.704612
"South Park: Bigger, Longer and Uncut (1999)",5.451100
Rounders (1998),5.298393
Blade Runner (1982),5.244951
Roger & Me (1989),5.191962
Gattaca (1997),5.183179
Ben-Hur (1959),5.130463
Rosencrantz and Guildenstern Are Dead (1990),5.087375
"Big Lebowski, The (1998)",5.038690


## 08. 파이썬 추천 시스템 패키지 - Surprise




### Surprise 패키지 소개


**Surprise** : 파이썬 기반의 추천 시스템 구축을 위한 전용 패키지

* 장점
  + 다양한 추천 알고리즘 쉽게 적용해 추천 시스템 구축 가능
  + Surprise의 핵심 API는 사이킷런의 핵심 API와 유사한 API명으로 작성

In [23]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162994 sha256=e949e19a53225b666ce2775af3a8498f351d32bf1bccbea13b412614c0e604ca
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


### Surprise를 이용한 추천 시스템 구축

* 추천 데이터를 학습용과 테스트용 데이터 세트로 분리 -> SVD 행렬 분해를 통한 잠재 요인 협업 필터링 수행

In [24]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

* 데이터 로딩은 Dataset 클래스 이용해서만 가능
* 주요 데이터가 로우 레벨 형태로 돼 있는 포맷의 데이터만 처리

In [25]:
# Surprise는 무비렌즈 사이트에서 제공하는 과거 버전의 데이터 세트 가져오는 API 제공
data = Dataset.load_builtin('ml-100k')
# 수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


* Surprise에 사용자-아이템 평점 데이터를 적용할 때 주의해야 할 점

-> Surprise는 자체적으로 로우 레벨의 데이터를 칼럼 레벨의 데이터로 변경하므로 원본인 로우 레벨의 사용자-아이템 평점 데이터를 데이터 세트로 적용해야 한다

* SVD로 잠재 요인 협업 필터링 수행

In [26]:
algo = SVD()
algo.fit(trainset)

* **test()** : 사용자-아이템 평점 데이터 세트 전체에 대해서 추천을 예측하는 메서드

* **predict()** : 개별 사용자와 영화에 대한 추천 평점 반환

#### test()

In [27]:
# 테스트 데이터 세트 전체에 대해 추천 영화 평점 데이터를 생성한 뒤에 최초 5개만 추출
predictions=algo.test(testset)
print('predictions type :', type(predictions), 'size :', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

predictions type : <class 'list'> size : 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.691991813467798, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.705707305506705, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.054191985450693, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.4039506443621894, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.2251276456948292, details={'was_impossible': False})]

=> Prediction 객체는 Surprise패키지에서 제공하는 데이터 타입.

=> 개별 사용자 아이디(uid), 영화 아이디(iid)와 실제 평점(r_ui) 정보에 기반해 Surprise의 추천 예측 평점(est) 데이터를 튜플 형태로 가져.

=> details 속성은 내부 처리 시 추천 예측을 할 수 없는 경우에 로그용으로 데이터를 남기는 데 사용. 'was_impossible'이 True면 예측값을 생성할 수 없는 데이터라는 의미

=> 속성에 접근하려면 객체명.uid와 같은 형식으로 가능

In [28]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.691991813467798),
 ('882', '291', 3.705707305506705),
 ('535', '507', 4.054191985450693)]

#### predict()

In [29]:
# 인자로 개별 사용자 아이디, 아이템 아이디 입력하면 추천 예측 평점을 포함한 정보 반환
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야함.
uid=str(196)
iid=str(302)
pred=algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.14   {'was_impossible': False}


* test()메서드는 입력 데이터 세트의 모든 사용자와 아이템 아이디에 대해서 predict()를 반복적으로 수행한 결과

In [30]:
# 테스트 데이터 세트 이용해 추천 예측 평점과 실제 평점과의 차이 평가
accuracy.rmse(predictions)

RMSE: 0.9492


0.9491839577460947

### Surprise 주요 모듈 소개

#### Dataset

* Surprise는 데이터 세트의 칼럼 순서가 사용자 아이디, 아이템 아이디, 평점 순으로 된 것만 적용 가능

>* Dataset.load_builtin(name='ml-100k') : 무비렌즈 아카이브 FTP 서버에서 무비렌즈 데이터 내려받기
>* Dataset.load_from_file(file_path, reader) : OS 파일에서 데이터 로딩할 때 사용
>* Dataset.load_from_df(df, reader) : 판다스의 DataFrame에서 데이터 로딩

#### OS 파일 데이터를 Surprise 데이터 세트로 로딩

* **Dataset.load_from_file** API 이용해 지정된 디렉터리에 있는 사용자-아이템 평점 데이터 로딩
  + 주의할 점 : 로딩되는 데이터 파일에 칼럼명을 가지는 헤더 문자열이 있어서는 안 된다
  + to_csv() 함수를 이용해 칼럼 헤더 삭제

In [32]:
import pandas as pd

ratings=pd.read_csv('ratings.csv')
# ratings_noh.csv 파일로 언로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성.
ratings.to_csv('ratings_noh.csv', index=False, header=False)

* **Reader 클래스**를 이용해 데이터 파일의 파싱 포맷을 정의해야 한다
  + 이 4개의 칼럼이 사용자 아이디, 아이템 아이디, 평점, 타임스탬프임을 로딩할 때 알려줘야 한다

* Reader 클래스의 생성자에 각 필드의 칼럼명과 칼럼 분리문자, 그리고 최소~최대 평점을 입력해 객체 생성 -> load_from_file()로 생성된 Reader 객체를 참조해 데이터 파일을 파싱하면서 로딩

In [33]:
from surprise import Reader

reader = Reader(line_format = 'user item rating timestamp', sep=',', rating_scale=(0.5,5))
data = Dataset.load_from_file('ratings_noh.csv', reader=reader)

* Reader 클래스의 주요 생성 파라미터
  + line_format(string) : 칼럼을 순서대로 나열. 입력된 문자열을 공백으로 분리해 칼럼으로 인식
  + sep(char) : 칼럼을 분리하는 분리자. 디폴트는 '\t'.
  + rating_scale(tuple, optional) : 평점 값의 최소~최대 평점 설정. 디폴트는 (1,5).

* SVD 행렬 분해 기법을 이용해 추천 예측
  + 잠재 요인 크기 K값을 나타내는 파라미터 n_factors 설정

In [34]:
trainset, testset=train_test_split(data, test_size=0.25, random_state=0)

#수행시마다 동일한 결과를 도출하기 위해 random_state 설정
algo=SVD(n_factors=50, random_state=0)

#학습 데이터 세트로 학습하고 테스트 데이터 세트로 평점 예측 후 rmse 평가
algo.fit(trainset)
predictions=algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

#### 판다스 DataFrame에서 Surprise 데이터 세트로 로딩

* **Dataset.load_from_df()** 이용
* 주의할 점 : DataFrame 역시 사용자 아이디, 아이템 아이디, 평점 칼럼 순서 지켜야 한다

In [35]:
import pandas as pd
from surprise import Reader, Dataset

ratings=pd.read_csv('ratings.csv')
reader=Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame에서 칼럼은 사용자 아이디, 아이템 아이디, 평점 순서 지켜야 한다
data=Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset=train_test_split(data, test_size=0.25, random_state=0)

algo=SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions=algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

### Surprise 추천 알고리즘 클래스

1. SVD : 행렬 분해를 통하 잠재 요인 협업 필터링을 위한 SVD 알고리즘
2. KNNBasic : 최근접 이웃 협업 필터링을 위한 KNN 알고리즘
3. BaselineOnly : 사용자 Bias와 아이템 Bias를 감안한 SGD 베이스라인 알고리즘

* Surprise SVD의 비용 함수는 사용자 베이스라인 편향성을 감안한 평점 예측에 Regularization 적용한 것
* SVD 클래스의 입력 파라미터
  + n_factors : 잠재 요인 K의 개수. 디폴트는 100. 커질수록 정확도가 높아질 수 있으나 과적합 문제 발생할 수도.
  + n_epochs : SGD 수행 시 반복 횟수. 디폴트는 20.
  + biased(bool) : 베이스라인 사용자 편향 적용 여부. 디폴트는 True


* SVD++ 알고리즘 성적 좋지만, 상대적으로 시간 너무 오래 걸려 사용 어려울 것. SVD와 k-NN Baseline 성능 평가 수치 좋아 -> Baseline 결합한 경우 성능 평가 수치 대폭 향상
  + Baseline : 각 개인이 평점을 부여하는 성향을 반영해 평점을 계산하는 방식

### 베이스라인 평점

**베이스라인 평점** : 개인의 성향을 반영해 아이템 평가에 편향성 요소를 반영하여 평점을 부과하는 것

* 보통 베이스라인 평점은 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수 공식으로 계산
  + 전체 평균 평점 = 모든 사용자의 아이템에 대한 평점을 평균한 값
  + 사용자 편향 점수 = 사용자별 아이템 평점 평균 값 - 전체 평균 평점
  + 아이템 편향 점수 = 아이템별 평점 평균 값 - 전체 평균 평점

### 교차 검증과 하이퍼 파라미터 튜닝

* Surprise는 교차 검증과 하이퍼 파라미터 튜닝을 위해 **cross_validate()**와 **GridSearchCV** 클래스 제공

* cross_validate() 이용해 데이터를 5개의 학습/검증 폴드 데이터 세트로 분리해 교차 검증 수행하고 RMSE, MAE로 성능 평가 진행
  + 인자로 알고리즘 객체, 데이터, 성능 평가 방법(measures), 폴드 데이터 세트 개수(cv) 입력

In [36]:
from surprise.model_selection import cross_validate

#판다스 df에서 surprise 데이터 세트로 데이터 로딩
ratings=pd.read_csv('ratings.csv')
reader=Reader(rating_scale=(0.5, 5.0))
data=Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo=SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8724  0.8729  0.8735  0.8726  0.8834  0.8750  0.0042  
MAE (testset)     0.6709  0.6706  0.6680  0.6725  0.6800  0.6724  0.0041  
Fit time          2.04    1.59    1.61    2.94    2.33    2.10    0.50    
Test time         0.12    0.15    0.11    0.18    0.17    0.15    0.03    


{'test_rmse': array([0.8724035 , 0.87288946, 0.87345643, 0.87262857, 0.88337819]),
 'test_mae': array([0.67085041, 0.67056714, 0.66798929, 0.67254846, 0.6800387 ]),
 'fit_time': (2.0425784587860107,
  1.5934979915618896,
  1.6082327365875244,
  2.9398014545440674,
  2.3345093727111816),
 'test_time': (0.11977744102478027,
  0.14691686630249023,
  0.1135861873626709,
  0.17524027824401855,
  0.1716141700744629)}

=> 폴드별 성능 평가 수치와 전체 폴드의 평균 성능 평가 수치 함께 보여줘

* GridSearchCV도 교차 검증을 통한 하이퍼 파라미터 최적화 수행
  + SVD의 경우 주로 점진적 하강 방식의 반복횟수를 지정하는 **n_epochs**와 SVD의 잠재 요인 K의 크기를 지정하는 **n_factors** 튜닝

In [37]:
from surprise.model_selection import GridSearchCV

#최적화할 파라미터를 딕셔너리 형태로 지정.
param_grid={'n_epochs':[20, 40, 60],
            'n_factors':[50, 100, 200]}

#CV를 3개 폴드 세트로 지정, 성능 평가를 rmse, mse로 수행하도록 GridSearchcv 구성
gs=GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

#최고 RMSE evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8783763629335608
{'n_epochs': 20, 'n_factors': 50}


### Surprise를 이용한 개인화 영화 추천 시스템 구축

Surprise 패키지로 학습된 추천 알고리즘을 기반으로 특정 사용자가 아직 평점을 매기지 않은 영화 중에서 개인 취향에 가장 적절한 영화 추천


In [38]:
# 다음 코드는 train_test_split()으로 분리되지 않는 데이터 세트에 fit()을 호출해 오류 발생

#data=Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
#algo=SVD(n_factors=50, random_state=0)
#algo.fit(data)

* 데이터 세트 전체를 학습 데이터로 사용하려면 **DatasetAutoFolds** 클래스 이용해 객체 생성 -> **build_full_trainset()** 메서드 호출

In [39]:
from surprise.dataset import DatasetAutoFolds

reader=Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성
data_folds=DatasetAutoFolds(ratings_file='ratings_noh.csv', reader=reader)

# 전체 데이터를 학습 데이터로 생성
trainset=data_folds.build_full_trainset()

* SVD 이용해 학습 수행

In [40]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

* userId 9가 아직 평점을 매기지 않은 영화를 movieId 42로 선정한 뒤 예측 평점 계산

In [41]:
# 영화에 대한 상세 속성 정보 DataFrame 로딩
movies=pd.read_csv('movies.csv')

# userId=9의 movieId 데이터를 추출해 movieId=42 데이터가 있는지 확인
movieIds=ratings[ratings['userId']==9]['movieId']

if movieIds[movieIds==42].count()==0:
  print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


* 추천 예상 평점은 predict() 메서드 이용
  + 입력해주는 값은 모두 문자열 값이어야 한다

In [42]:
uid=str(9)
iid=str(42)

pred=algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


* 사용자가 평점을 매기지 않은 전체 영화를 추출한 뒤에 예측 평점 순으로 영화 추천

In [43]:
# 사용자가 아직 평점을 매기지 않은 영화 정보 반환
def get_unseen_surprise(ratings, movies, userId):
  #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
  seen_movies=ratings[ratings['userId']==userId]['movieId'].tolist()

  #모든 영화의 movieId를 리스트로 생성
  total_movies=movies['movieId'].tolist()

  #모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트로 생성
  unseen_movies=[movie for movie in total_movies if movie not in seen_movies]
  print('평점 매긴 영화 수:', len(seen_movies), '추천 대상 영화 수:', len(unseen_movies), '전체 영화 수:', len(total_movies))

  return unseen_movies

unseen_movies=get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742


* 학습된 추천 알고리즘 클래스의 SVD를 이용해 높은 예측 평점을 가진 순으로 영화 추천
  + 함수 생성 - 인자로 학습이 완료된 추천 알고리즘 객체, 추천 대상 사용자 아이디, 추천 대상 영화의 리스트 객체, 추천 상위 N개 개수

In [44]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
  #알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list로 저장
  predictions=[algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

  #predictions list 객체는 surprise의 predictions 객체를 원소로 갖고 있다.
  # [Prediction(uid='9', iid='1', est=3.69), Predictions(uid='9', iid='2', est=2.98),,,]

  #이를 est 값으로 정렬하기 위해 아리애 sortkey_est 함수 정의
  #sortdey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행
  def sortkey_est(pred):
    return pred.est

  #sortkey_est() 반환값의 내림차순으로 정렬 수행하고 top_n개의 최상위 값 추출
  predictions.sort(key=sortkey_est, reverse=True)
  top_predictions=predictions[:top_n]

  #top_n 으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
  top_movie_ids=[int(pred.iid) for pred in top_predictions]
  top_movie_rating=[pred.est for pred in top_predictions]
  top_movie_titles=movies[movies.movieId.isin(top_movie_ids)]['title']
  top_movie_preds=[ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]

  return top_movie_preds

unseen_movies=get_unseen_surprise(ratings, movies, 9)
top_movie_preds=recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('### top-10 추천 영화 리스트 ###')
for top_movie in top_movie_preds:
  print(top_movie[1], ':', top_movie[2])

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742
### top-10 추천 영화 리스트 ###
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122657
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957


## 09. 정리


---

**추천 시스템** : 기업 애플리케이션에서 매우 중요한 위치 차지

1. 콘텐츠 기반 필터링 : 아이템을 구성하는 여러 가지 콘텐츠 중 사용자가 좋아하는 콘텐츠를 필터링하여 이에 맞는 아이템을 추천하는 방식

2. 협업 필터링
  * 최근접 이웃 협업 필터링 - 사용자 기반(사용자-사용자)과 아이템 기반(아이템-아이템)
  * 잠재 요인 협업 필터링

* Surprise : 파이썬의 추천 시스템 패키지